This program reads in a non-profit company's transaction register and provides total annual spending for each program in the major categories including salaries & benefits (S&B), non-personnel, administrative, and client costs. In addition, projected total annual spending estimates are provided for each department.  

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

Read and Clean Data 

In [2]:
df_trans = pd.read_csv("Trans_Nov.csv") #Read in company transaction register 
df_chart = pd.read_csv("Chart_Accounts.csv") #Read in company expense code legend

Clean and Modify df_chart

In [3]:
df_chart.head()

,REVENUE,Unnamed: 1,Unnamed: 2,40000,49999,Unnamed: 5,Unnamed: 6,INACTIVE,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Donated Goods Retail,40000.0,40999.0,Map To,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100,NaN,Retail Sales - Rack Goods,40100.0,NaN,477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100,NaN,Retail Sales - Bin Goods,40200.0,NaN,478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100,NaN,Retail Sales - Furniture,40300.0,NaN,483,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#clean chart of accounts
df_chart_new = df_chart.rename(columns={"Unnamed: 2":"Expense_Code", "40000":"Code"}) 
df_chart_new.dropna(how='all', inplace=True)#drop null values
df_chart_new = df_chart_new[["Expense_Code", "Code"]] #we only the listed fields
df_chart_new.reset_index(drop=True, inplace = True)#reset index
df_chart_new #All prepared and in the proper structure

,Expense_Code,Code
0,Donated Goods Retail,40000.0
1,Retail Sales - Rack Goods,40100.0
2,Retail Sales - Bin Goods,40200.0
3,Retail Sales - Furniture,40300.0
4,Retail Sales - E & M,40400.0
...,...,...
150,Cost of Compensation Allocation,79000.0
151,Allocated Indirect Admin Overhead,79100.0
152,Non Cash and Non Operating,90000.0
153,Depreciation & Amortization,90000.0


Clean and Modify df_trans

In [5]:
#Modify, clean and use only needed fields
df_trans_new = df_trans[["Account", "Department.1", "Location.1", " Balance "]] 
df_trans_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4681 entries, 0 to 4680
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Account       4681 non-null   int64 
 1   Department.1  4681 non-null   object
 2   Location.1    3740 non-null   object
 3    Balance      4681 non-null   object
dtypes: int64(1), object(3)
memory usage: 146.4+ KB


In [6]:
df_trans_new[' Balance '] = df_trans_new[' Balance '].str.replace("(","-")#make balance a float and remove () that indicate negative number
df_trans_new[' Balance '] = df_trans_new[' Balance '].str.replace(')', "") #[' Balance '].str.replace(")","")
df_trans_new[" Balance "] = df_trans_new[" Balance "].str.replace(",","")
df_trans_new[" Balance "] = df_trans_new[" Balance "].str.strip() #remove extra spaces
df_trans_new[" Balance "]= df_trans_new[" Balance "].astype(float) #change data type to float

<ipython-input-6-9216fda9c979>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans_new[' Balance '] = df_trans_new[' Balance '].str.replace("(","-")#make balance a float and remove () that indicate negative number
<ipython-input-6-9216fda9c979>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans_new[' Balance '] = df_trans_new[' Balance '].str.replace(')', "") #[' Balance '].str.replace(")","")
<ipython-input-6-9216fda9c979>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [7]:
df_trans_new.insert(2, "Dept_Loc", value=2) #insert new column

In [8]:
df_trans_new["Location.1"] = df_trans_new["Location.1"].astype(str) #set as string data type

<ipython-input-8-dba5f30ae3f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans_new["Location.1"] = df_trans_new["Location.1"].astype(str) #set as string data type


In [9]:
#combine Department.1 and Location.1 into newly created Dept_Loc column. 
df_trans_new["Dept_Loc"] = df_trans_new["Department.1"] + "-"+df_trans_new["Location.1"]

<ipython-input-9-78dd25f55dfd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trans_new["Dept_Loc"] = df_trans_new["Department.1"] + "-"+df_trans_new["Location.1"]


In [10]:
df_trans_final = df_trans_new.drop(columns=(["Department.1", "Location.1"])) #drop fields 
df_trans_final.head() #Dataframe is complete

,Account,Dept_Loc,Balance
0,61100,5659-100,3.88
1,61100,5671-200,-42.99
2,61100,5672-200,-24.87
3,61100,5672-200,43.98
4,61100,5692-200,0.57


Create Expense Code Lists

In [11]:
#Putting company expense codes into lists based off the general category they fall under
salaries_list = [60220,60230,61010,61020,61030,61040,61100,61190,61310,61320,61330,61395,79000] #salaries, wage and benefits,taxes
client_wages_list = [60100,60210] #client wages expense codes
Admin = [79100] #Administrative expense codes
Revenue = [43100,43200,43300,43400,43500,43600,43700,44600,44950] #Revenue expense codes
comb_list = salaries_list + client_wages_list + Admin + Revenue #This later will be used to find non-personnel expenses

#Department or program numbers below
Departments = ['5657-100', '5657-200', '5658-100','5691-100', '5692-100', '5692-200', '5692-500', '5711-200', '5659-100']


Finding Total Expenses for Each Program and General Category (S&B, Nonpersonnel, Admin, Client Wages)

In [12]:
#Create lists for each general category of total expenses 
wages = []
non_person = []
Administration = []
Client = []

#This for loop will filter the dataframe by program department code and provide totals for each general category
#The resuls will eventually be combined into a dataframe.
for i in Departments: #for loop that will go through each department and calculate total expenses
    program = df_trans_final[df_trans_final["Dept_Loc"] == i] #filters dataframe for spec. dept.

    sb_owp = program[program["Account"].isin(salaries_list)] #filters for only S&B expense codes
    elim_client = sb_owp[" Balance "].sum() #sums total S&B expenses including client FICA and WC
    Cli_wages = program[program["Account"].isin(client_wages_list)] #Client Wages 
    wages.append(elim_client - (Cli_wages[" Balance "].sum())*.1675) #Eliminate Client FICA & WC from S&B

    non_personnel = program[~program["Account"].isin(comb_list)] #any expenses not in comb_list default to non-personnel
    non_person.append(non_personnel[" Balance "].sum())

    Administ = program[program["Account"].isin(Admin)] #Admin
    Administration.append(Administ[" Balance "].sum())

    Cli_wages = program[program["Account"].isin(client_wages_list)] #Client Wages & FICA
    Client.append((Cli_wages[" Balance "].sum())*1.1675)


In [13]:
#Convert all lists above to Series
wages = pd.Series(wages)
non_person = pd.Series(non_person)
Administration = pd.Series(Administration)
Client = pd.Series(Client)
total = pd.Series()
Departments = pd.Series(Departments)

<ipython-input-13-eb875d030621>:6: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  total = pd.Series()


In [14]:
#concat Series above together to create new dataframe
dashboard = pd.concat([Departments, wages, non_person, Administration, Client, total], axis=1)
dashboard.rename(columns={0:"DEPT", 1:"S&B", 2:"NON_PERSONNEL",3:"ADMINISTRATION",4:"CLIENT", 5:"TOTAL_EXPENSES"}, inplace=True)
dashboard

,DEPT,S&B,NON_PERSONNEL,ADMINISTRATION,CLIENT,TOTAL_EXPENSES
0,5657-100,197500.481350,17551.32,19672.39,130216.388650,NaN
1,5657-200,78992.314375,4166.99,8375.55,2074.355625,NaN
2,5658-100,3076.177450,115.48,1179.40,9133.422550,NaN
3,5691-100,0.000000,560.86,0.00,0.000000,NaN
4,5692-100,1491.032700,138.54,0.00,1574.677300,NaN
5,5692-200,437.201500,189.45,0.00,250.078500,NaN
6,5692-500,3331.007250,2921.55,0.00,7452.502750,NaN
7,5711-200,11083.860000,212.84,1697.69,0.000000,NaN
8,5659-100,4685.640000,1871.20,655.69,0.000000,NaN


In [15]:
#TOTAL_EXPENSES column is based off columns S&B - CLIENT added together...
dashboard["TOTAL_EXPENSES"] = dashboard["S&B"] + dashboard["NON_PERSONNEL"] + dashboard["ADMINISTRATION"] + dashboard["CLIENT"]
dashboard

,DEPT,S&B,NON_PERSONNEL,ADMINISTRATION,CLIENT,TOTAL_EXPENSES
0,5657-100,197500.481350,17551.32,19672.39,130216.388650,364940.58
1,5657-200,78992.314375,4166.99,8375.55,2074.355625,93609.21
2,5658-100,3076.177450,115.48,1179.40,9133.422550,13504.48
3,5691-100,0.000000,560.86,0.00,0.000000,560.86
4,5692-100,1491.032700,138.54,0.00,1574.677300,3204.25
5,5692-200,437.201500,189.45,0.00,250.078500,876.73
6,5692-500,3331.007250,2921.55,0.00,7452.502750,13705.06
7,5711-200,11083.860000,212.84,1697.69,0.000000,12994.39
8,5659-100,4685.640000,1871.20,655.69,0.000000,7212.53


Add Annual Spending Projection Column

In [16]:
#User will enter the date of the last transaction from the transaction register 
end_date = pd.to_datetime(input("Enter ending date of transaction log: ")) #input ending data of transaction log
end_program_year = pd.to_datetime('06/30/2022') #This is the ending date of the program year

weeks_left = end_program_year - end_date #weeks left in program year
remaining_weeks = weeks_left / np.timedelta64(1, "W") #convert timedelta to float

weeks_down = 52-remaining_weeks #Weeks that have already happened

Enter ending date of transaction log: 09/30/2021


In [17]:
#Takes average weekly spending and multiplys by 52 to get projection in for year. 
#An ANNUAL_PROJECTIONS column is created for projected annaul spending
dashboard = dashboard.assign(ANNUAL_PROJECTIONS=lambda x:(dashboard["TOTAL_EXPENSES"]/weeks_down)*52)

In [18]:
dashboard 
#Final report that shows total spending by Department in each general spending category.
#The ANNUAL_PROJECTIONS column is the total spending estimated to take place throughout the year. 

,DEPT,S&B,NON_PERSONNEL,ADMINISTRATION,CLIENT,TOTAL_EXPENSES,ANNUAL_PROJECTIONS
0,5657-100,197500.481350,17551.32,19672.39,130216.388650,364940.58,1459762.32
1,5657-200,78992.314375,4166.99,8375.55,2074.355625,93609.21,374436.84
2,5658-100,3076.177450,115.48,1179.40,9133.422550,13504.48,54017.92
3,5691-100,0.000000,560.86,0.00,0.000000,560.86,2243.44
4,5692-100,1491.032700,138.54,0.00,1574.677300,3204.25,12817.00
5,5692-200,437.201500,189.45,0.00,250.078500,876.73,3506.92
6,5692-500,3331.007250,2921.55,0.00,7452.502750,13705.06,54820.24
7,5711-200,11083.860000,212.84,1697.69,0.000000,12994.39,51977.56
8,5659-100,4685.640000,1871.20,655.69,0.000000,7212.53,28850.12
